In [5]:
import subprocess

input_file = "prepa.mp4"
output_file = "clip.mp4"
start_time = "00:00:00"
duration = "00:00:10"

# Use FFmpeg to extract the clip
subprocess.call(["ffmpeg", "-i", input_file, "-ss", start_time, "-t", duration, "-c", "copy", output_file])

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

1

In [1]:
# !export XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/local/cuda/
!export CUDA_DIR=/usr/lib/cuda/
# !export LD_LIBRARY_PATH=/usr/local/cuda-11.8/lib64:/home/pedro/miniconda3/envs/ml2/lib/
# !export TF_GPU_ALLOCATOR=cuda_malloc_async

In [9]:
import cv2
import numpy as np
import tensorflow as tf
import time
from IPython import display # If using IPython, Colab or Jupyter

# Load the video
cap = cv2.VideoCapture("clip.mp4")

# Get video information
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

# Load the TensorFlow model
model = tf.keras.models.load_model('./saved_models/denoise_autoencoder')

print(f"fps={fps} fourcc={fourcc}")

# Initialize the output video writer
out = cv2.VideoWriter("output_video.mp4", fourcc, fps/2, (1080, 720))

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    
    # Break the loop if the video has ended
    if not ret:
        break

    # Preprocess the frame
#     frame = cv2.resize(frame, (224, 224))
#     frame = frame / 255.0
    frame = frame.astype("float32")
    frame = np.expand_dims(frame, axis=0)
    
    # Pass the frame through the model
    with open(os.devnull, "w") as f:
        prediction = model.predict(frame)
    display.clear_output()

    # Postprocess the prediction
    prediction = np.squeeze(prediction, axis=0)
    prediction = np.round(prediction).astype("uint8")

    # Write the processed frame to the output video
    out.write(prediction)
    time.sleep(1.0 / fps)

# Release the video capture and writer objects
cap.release()
out.release()